### Penny in-class exercise

 Suppose we drop a quarter from the Empire State Building and find that its flight time is 19.1 seconds.  Use this measurement to estimate terminal velocity and coefficient of drag.

You can get the relevant dimensions of a quarter from <https://en.wikipedia.org/wiki/Quarter_(United_States_coin)>.

1. Create a `params` object with new values of `mass` and `diameter`. We don't know `v_term`, so we'll start with the initial guess 18 m/s.

2. Use `make_system` to create a `system` object.  

3. Call `run_solve_ivp` to simulate the system.  How does the flight time of the simulation compare to the measurement?

4. Try a few different values of `v_term` and see if you can get the simulated flight time close to 19.1 seconds.

5. Write an error function and use `root_scalar` to improve your estimate (See chapter 4.1 for help).

6. Use your best estimate of `v_term` to compute `C_d`.

Note: I fabricated the "observed" flight time, so don't take the results of this exercise too seriously.

In [ ]:
# Create a new params object and make a system
params_q = params.copy()
params_q.update(dict(mass = 0.00567,
                     diameter = 0.02426,
                     v_term = 21.7))
system_q = make_system(params_q)

In [ ]:
# Test with assumed terminal velocity; iterate manually to find approximate
# value for v_term
results_q, details = run_solve_ivp(system_q, slope_func,
                                 events=event_func)
results_q.index[-1]

In [ ]:
# Write an error_func 

def error_func_coin(v_term, system):
    system['v_term'] = v_term
    
    rho, area = system['rho'], system['area']
    mass, g = system['mass'], system['g']
    
    system['C_d'] = (2 * mass * g / 
       (rho * area * v_term**2))
    
    results, details = run_solve_ivp(system, slope_func,
                                 events=event_func)
    return results.index[-1] - 19.1


In [ ]:
# Run root scalar to find the best v_term

res = root_scalar(error_func_coin, system_q, bracket=[10, 30])
res

In [ ]:
# Check your answer nad Calculate the best C_d

params_q_final = params_q.copy()
params_q_final.update(dict(v_term = 21.688400))
system_q_final = make_system(params_q_final)
results_q_final, details = run_solve_ivp(system_q_final, slope_func,
                                 events=event_func)
print('Time:', results_q_final.index[-1])
print('Coefficient of Drag:', system_q_final['C_d'])
